#Import

In [ ]:
%pip install -q web3

In [ ]:
%pip install -q pandarallel

In [1]:
# from web3 import Web3
import pandas as pd
pd.set_option('display.max_rows', 100)
from tqdm import tqdm
import ast #eval() to interpret code
import json
import math
import numpy as np
import time
# from concurrent.futures import ThreadPoolExecutor

In [2]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
#Query
'''
SELECT
        block_timestamp,
        block_number,
        transaction_index,
        trace_address,
        transaction_hash,
        input,
        ARRAY_TO_STRING(
                ARRAY(
                        SELECT CHR(CAST(num AS INT64)) FROM UNNEST(SPLIT(trace_address, ',')) AS num
                ),
                ","
        ) as trace_addr_str
FROM  `bigquery-public-data.crypto_ethereum.traces`
WHERE DATE(block_timestamp) >= "2019-11-12"
        and trace_address IS NOT NULL
        and call_type = "call"
        and to_address = LOWER("0x35D1b3F3D7966A1DFe207aa4514C12a259A0492B")
        and status = 1
ORDER BY CAST(block_number as INT64), CAST(transaction_index as INT64), trace_addr_str

'''

''

#Read file data as of Jul 2023

In [3]:
df = pd.read_csv('Data_Jul_2023.csv')
df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
df

,order,block_timestamp,block_number,transaction_index,trace_address,transaction_hash,input,trace_addr_str
0,0,2019-11-13 19:16:32 UTC,8928152,66,"0,1",0xa39edcdeb7310150d7be44b8da94e5043d1cf8600d59...,0x65fae35e000000000000000000000000baa65281c2fa...,NaN
1,1,2019-11-13 19:16:32 UTC,8928152,66,"0,2",0xa39edcdeb7310150d7be44b8da94e5043d1cf8600d59...,0x9c52a7f1000000000000000000000000403689148fa9...,NaN
2,2,2019-11-13 19:16:32 UTC,8928152,66,2,0xa39edcdeb7310150d7be44b8da94e5043d1cf8600d59...,0x65fae35e00000000000000000000000065c79fcb50ca...,
3,3,2019-11-13 19:18:41 UTC,8928160,33,2,0xaac1fe54d617a879bb05bec28b1dd4159f3ad1ba1e67...,0x65fae35e00000000000000000000000019c0976f590d...,
4,4,2019-11-13 19:18:41 UTC,8928160,33,3,0xaac1fe54d617a879bb05bec28b1dd4159f3ad1ba1e67...,0x65fae35e000000000000000000000000197e90f9fad8...,
...,...,...,...,...,...,...,...,...
4466875,4466875,2023-07-31 23:49:35 UTC,17816381,9,"1,0,1,0,1,1,1,1,1",0x26c9d1e2d7f72436488525cb124c9d1ba9a4e3e9a81a...,0x7608870350534d2d555344432d410000000000000000...,","
4466876,4466876,2023-07-31 23:49:35 UTC,17816381,9,"1,0,1,0,1,1,1,1,2",0x26c9d1e2d7f72436488525cb124c9d1ba9a4e3e9a81a...,0xbb35783b00000000000000000000000089b78cfa322f...,","
4466877,4466877,2023-07-31 23:49:35 UTC,17816381,9,"1,0,1,0,1,1,1,1,3,0",0x26c9d1e2d7f72436488525cb124c9d1ba9a4e3e9a81a...,0xbb35783b00000000000000000000000089b78cfa322f...,","
4466878,4466878,2023-07-31 23:55:59 UTC,17816413,8,"0,1,0,0,0,8,1",0xe799722397920e4c9a7d5267bec2650ea8846220880d...,0x1a0b287e554e495632444149555344432d4100000000...,NaN


In [4]:
df.columns

Index(['order', 'block_timestamp', 'block_number', 'transaction_index',
       'trace_address', 'transaction_hash', 'input', 'trace_addr_str'],
      dtype='object')

In [7]:
cols = ['transaction_hash', 'input']
df[cols].head(4)

,transaction_hash,input
0,0xa39edcdeb7310150d7be44b8da94e5043d1cf8600d59...,0x65fae35e000000000000000000000000baa65281c2fa...
1,0xa39edcdeb7310150d7be44b8da94e5043d1cf8600d59...,0x9c52a7f1000000000000000000000000403689148fa9...
2,0xa39edcdeb7310150d7be44b8da94e5043d1cf8600d59...,0x65fae35e00000000000000000000000065c79fcb50ca...
3,0xaac1fe54d617a879bb05bec28b1dd4159f3ad1ba1e67...,0x65fae35e00000000000000000000000019c0976f590d...


In [8]:
df['input'].iloc[0]

'0x65fae35e000000000000000000000000baa65281c2fa2baacb2cb550ba051525a480d3f4'

In [ ]:
10,6,10
10,6,6

Decode hidden Data

In [6]:
abi_vat = open("MCD-Vat.txt", 'r').read()
w3 = Web3()
contract = w3.eth.contract(abi=abi_vat)

In [7]:
def decode(row):
  result = contract.decode_function_input(row['input']) #tuple
  index = ['timestamp', 'order_trx', 'transaction_hash', 'fn_name', 'args']
  fn_name = result[0].fn_name #function name
  fn_args = result[1]
  txn_hash = row['transaction_hash']
  block_timestamp = row['block_timestamp']
  order = row['order']
  return pd.Series(data=(block_timestamp, order, txn_hash, fn_name, fn_args), index=index)

In [11]:
%%time
results = df.parallel_apply(decode, axis=1)
results

CPU times: user 53.3 s, sys: 12.1 s, total: 1min 5s
Wall time: 35min 8s


,timestamp,order_trx,transaction_hash,fn_name,args
0,2019-11-13 19:16:32 UTC,0,0xa39edcdeb7310150d7be44b8da94e5043d1cf8600d59...,rely,{'usr': '0xbaa65281c2FA2baAcb2cb550BA051525A48...
1,2019-11-13 19:16:32 UTC,1,0xa39edcdeb7310150d7be44b8da94e5043d1cf8600d59...,deny,{'usr': '0x403689148fA98a5a6fDcC0b984914Ae968d...
2,2019-11-13 19:16:32 UTC,2,0xa39edcdeb7310150d7be44b8da94e5043d1cf8600d59...,rely,{'usr': '0x65C79fcB50Ca1594B025960e539eD7A9a6D...
3,2019-11-13 19:18:41 UTC,3,0xaac1fe54d617a879bb05bec28b1dd4159f3ad1ba1e67...,rely,{'usr': '0x19c0976f590D67707E62397C87829d896Dc...
4,2019-11-13 19:18:41 UTC,4,0xaac1fe54d617a879bb05bec28b1dd4159f3ad1ba1e67...,rely,{'usr': '0x197E90f9FAD81970bA7976f33CbD77088E5...
...,...,...,...,...,...
4466875,2023-07-31 23:49:35 UTC,4466875,0x26c9d1e2d7f72436488525cb124c9d1ba9a4e3e9a81a...,frob,{'i': b'PSM-USDC-A\x00\x00\x00\x00\x00\x00\x00...
4466876,2023-07-31 23:49:35 UTC,4466876,0x26c9d1e2d7f72436488525cb124c9d1ba9a4e3e9a81a...,move,{'src': '0x89B78CfA322F6C5dE0aBcEecab66Aee4539...
4466877,2023-07-31 23:49:35 UTC,4466877,0x26c9d1e2d7f72436488525cb124c9d1ba9a4e3e9a81a...,move,{'src': '0x89B78CfA322F6C5dE0aBcEecab66Aee4539...
4466878,2023-07-31 23:55:59 UTC,4466878,0xe799722397920e4c9a7d5267bec2650ea8846220880d...,file,{'ilk': b'UNIV2DAIUSDC-A\x00\x00\x00\x00\x00\x...


In [12]:
results.to_csv('decode_Jul_2023.csv')

In [14]:
results[results['transaction_hash']=='0xf568d75a9d16b43e4703d06d63f9626fa284ddceaf360eb790457512086b4bf6']

,timestamp,order_trx,transaction_hash,fn_name,args
3760809,2022-04-25 05:40:34 UTC,3760809,0xf568d75a9d16b43e4703d06d63f9626fa284ddceaf36...,move,{'src': '0x9759A6Ac90977b93B58547b4A71c78317f3...
3760810,2022-04-25 05:40:34 UTC,3760810,0xf568d75a9d16b43e4703d06d63f9626fa284ddceaf36...,frob,{'i': b'ETH-A\x00\x00\x00\x00\x00\x00\x00\x00\...
3760811,2022-04-25 05:40:34 UTC,3760811,0xf568d75a9d16b43e4703d06d63f9626fa284ddceaf36...,frob,{'i': b'ETH-A\x00\x00\x00\x00\x00\x00\x00\x00\...
3760812,2022-04-25 05:40:34 UTC,3760812,0xf568d75a9d16b43e4703d06d63f9626fa284ddceaf36...,flux,{'ilk': b'ETH-A\x00\x00\x00\x00\x00\x00\x00\x0...
3760813,2022-04-25 05:40:34 UTC,3760813,0xf568d75a9d16b43e4703d06d63f9626fa284ddceaf36...,slip,{'ilk': b'ETH-A\x00\x00\x00\x00\x00\x00\x00\x0...


#-----------End preprocessing01---------